<a href="https://colab.research.google.com/github/agistarakha/RasainApp-ml/blob/develop/Model%20Project/model_project_bangkit_10Class(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Prepare the Dataset

Import library

In [ ]:
#!pip install tensorflowjs

In [ ]:
import tensorflow as tf
import subprocess

Copy dataset from drive

In [ ]:
cp -R /content/drive/MyDrive/indo_food_datasets/jadi/food-dataset-500 /content/

Unzip file

In [ ]:
import zipfile

# Extract the archive
local_zip = './food-dataset-500.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/food-dataset')
zip_ref.close()

# local_zip = './rps-test-set.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('tmp/rps-test')
# zip_ref.close()

Delete unused dataset

In [ ]:
food_classes = ['soto','pepes', 'mendoan', 'martabak', 'lumpia']

for food_class in food_classes:
  subprocess.run(["rm", "-rf", "/content/food-dataset-500/test/"+food_class])
  subprocess.run(["rm", "-rf", "/content/food-dataset-500/train/"+food_class])

## Model

Build Model Layer

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Print the model summary
model.summary()

Compile Model

In [ ]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.Adam() , metrics=['accuracy'])

## Prepare the ImageDataGenerator

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/food-dataset-500/train"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/content/food-dataset-500/test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
  #batch_size=126
)

## Train the model and evaluate the results


Define Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching 60 percent accuracy

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check accuracy
    # if(logs.get('loss') < 0.4):

    #   # Stop if threshold is met
    #   print("\nLoss is lower than 0.4 so cancelling training!")
    #   self.model.stop_training = True
    if(logs.get('val_accuracy') > 0.75 and logs.get('accuracy') > 0.75):
      # Stop if threshold is met
      print("\nVal_accuracy is higher than 0.8 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

Train Model

In [ ]:
history = model.fit(train_generator, epochs=50, validation_data = validation_generator, verbose = 1, validation_steps=3, callbacks=[callbacks])

Plot the Graph history

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

# Model Prediction

In [ ]:
## CODE BLOCK FOR NON-SAFARI BROWSERS
## SAFARI USERS: PLEASE SKIP THIS BLOCK AND RUN THE NEXT ONE INSTEAD

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

In [ ]:
import time
saved_model_path = "./saved_model/{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./saved_model/js/

In [ ]:
!zip -r model.zip saved_model

## Finish